## Electricity usage in Eagle 103
Try forecasting on all buildings (separately) from one site with daily mean.

In [1]:
DATAPATH=''
try:
    # On Google Drive, set path to my drive / data directory.
    from google.colab import drive
    IN_COLAB = True
    PATH='/content/drive/'
    drive.mount(PATH)
    DATAPATH=PATH+'My Drive/data/'  # must end in "/"
except:
    # On home computer, set path to local data directory.
    IN_COLAB = False
    DATAPATH='data:/'  # must end in "/"

ZIP_FILE='BuildingData.zip'
ZIP_PATH = DATAPATH+ZIP_FILE
ELECT_FILE='electricity.csv'
WEATHER_FILE='weather.csv'
MODEL_FILE='Model'  # will be used later to save models

In [2]:
from os import listdir
import csv
from zipfile import ZipFile
import numpy as np
import pandas as pd
from pandas.plotting import autocorrelation_plot
from scipy import stats  # mode

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
from matplotlib import colors
mycmap = colors.ListedColormap(['red','blue'])  # list color for label 0 then 1
np.set_printoptions(precision=2)

In [3]:
def read_zip_to_panda(zip_filename,csv_filename):
    zip_handle = ZipFile(zip_filename)
    csv_handle = zip_handle.open(csv_filename)
    panda = pd.read_csv(csv_handle)
    return panda
def fix_date_type(panda):
    # Convert the given timestamp column to the pandas datetime data type.
    panda['timestamp'] = pd.to_datetime(panda['timestamp'], infer_datetime_format = True)
    indexed = panda.set_index(['timestamp'])
    return indexed
def get_site_timeseries(panda,site):
    # Assume the panda dataframe has a datetime column.
    # (If not, call fix_date_type() before this.)
    # Extract the timeseries for one site.
    # Convert the datetime column to a DatetimeIndex.
    site_df = panda[panda['site_id']==site]
    temp_col = site_df['date']
    temp_val = temp_col.values
    temp_ndx = pd.DatetimeIndex(temp_val)
    dropped = site_df.drop('date',axis=1)
    panda = dropped.set_index(temp_ndx)
    return panda

In [4]:
SITE = 'Eagle'
METER = 'Electricty'
BLDG = 'Eagle_education_Peter'
wet_df = read_zip_to_panda(ZIP_PATH,WEATHER_FILE)
wet_df = fix_date_type(wet_df)
elec_df = read_zip_to_panda(ZIP_PATH,ELECT_FILE)
elec_df = fix_date_type(elec_df)
site_specific_weather = wet_df.loc[wet_df['site_id'] == SITE]
bldg_specific_elect = elec_df[[BLDG]]
all_buildings = [x for x in elec_df.columns if x.startswith(SITE)] 

In [5]:
cors = []
MAX_BAD = 500 # correlation is higher in buildings without so many NaN and 0
for BLDG in all_buildings:
    bldg_specific_elect = elec_df[[BLDG]]    
    one_bldg_df = pd.concat([bldg_specific_elect,site_specific_weather],axis=1)
    one_bldg_df = one_bldg_df.drop(['site_id'],axis=1)
    one_bldg_df = one_bldg_df.rename(columns={BLDG : METER})
    one_bldg_df = one_bldg_df.fillna(0)
    bad = one_bldg_df[METER].isin([0]).sum()
    one_bldg_df = one_bldg_df.resample("1D").mean()  # replace hourly with daily mean
    if bad<=500:
        mean = one_bldg_df[METER].mean()
        cor = one_bldg_df.corr().iloc[0][3]
        # Linear Regression
        X = one_bldg_df.drop(METER,axis=1)
        y = one_bldg_df[METER]
        split = len(X)//2 # 900
        X_train = X.iloc[0:split]
        y_train = y.iloc[0:split]
        X_test = X.iloc[split:]
        y_test = y.iloc[split:]
        linreg = LinearRegression()
        linreg.fit(X_train,y_train)
        y_pred = linreg.predict(X_test)
        rmse = mean_squared_error(y_test,y_pred,squared=False)
        cors.append([cor,mean,rmse,rmse/mean,BLDG])

print("dew temp corr, dew temp mean, lin reg RMSE, RMSE/mean, BLDG")
for cor in sorted(cors):
    print("%7.4f %10.2f %10.2f %5.2f   %s"%(cor[0],cor[1],cor[2],cor[3],cor[4]))    

dew temp corr, dew temp mean, lin reg RMSE, RMSE/mean, BLDG
-0.9013      29.20       6.96  0.24   Eagle_public_Henry
-0.7136     187.01      25.46  0.14   Eagle_education_Lino
-0.6833     294.61      38.40  0.13   Eagle_education_Shanna
-0.3922     444.27      99.45  0.22   Eagle_assembly_Ian
-0.3916      47.80      12.36  0.26   Eagle_office_Efrain
-0.3603      47.95       9.21  0.19   Eagle_office_Elvis
-0.3253      65.38      16.76  0.26   Eagle_office_Donovan
-0.3250     146.60      34.28  0.23   Eagle_public_Minnie
-0.3078     907.82      47.91  0.05   Eagle_education_Brooke
-0.2901      60.09      16.86  0.28   Eagle_office_Freida
-0.2804     304.14      31.48  0.10   Eagle_health_Trisha
-0.2514     126.85      25.12  0.20   Eagle_education_Wesley
-0.2489     348.20      27.58  0.08   Eagle_office_Francis
-0.2463      54.75      13.75  0.25   Eagle_lodging_Andy
-0.2440      44.21       8.71  0.20   Eagle_office_Jeff
-0.2013      78.39      12.12  0.15   Eagle_lodging_Stephanie
-0